# Lifespan Events

In [ ]:
# | hide

from IPython.display import Markdown as md

Did you know that you can define some special code that runs before and after your Kafka application? This code will be executed just once, but it covers the whole lifespan of your app! :rocket:

Lets break it down:

You can define logic (code) that should be executed before the application starts up. This is like a warm-up for your app, getting it ready to consume and produce messages.

Similarly, you can define logic (code) that should be executed when the application is shutting down. This is like a cool-down for your app, making sure everything is properly closed and cleaned up.

By executing code before consuming and after producing, you cover the entire lifecycle of your application :tada:

This is super handy for setting up shared resources that are needed across consumers and producers, like a database connection pool or a machine learning model. And the best part? You can clean up these resources when the app is shutting down!

So lets give it a try and see how it can make your Kafka app even more awesome! :muscle:

## Lifespan example - Iris prediction model

Let's dive into an example to see how you can leverage the lifecycle handler to solve a common use case. Imagine that you have some machine learning models that need to consume incoming messages and produce response/prediction messages. These models are shared among consumers and producers, which means you don't want to load them for every message.

Here's where the lifecycle handler comes to the rescue! By loading the model before the messages are consumed and produced, but only right before the application starts receiving messages, you can ensure that the model is ready to use without compromising the performance of your tests. In the upcoming sections, we'll walk you through how to initialize an Iris species prediction model and use it in your developed application.

### Lifespan

You can define this startup and shutdown logic using the lifespan parameter of the FastKafka app, and an async context manager.

Let's start with an example and then see it in detail.

We create an async function lifespan() with yield like this:

In [ ]:
lifespan = """from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression

ml_models = {}

@asynccontextmanager
async def lifespan():
    # Load the ML model
    X, y = load_iris(return_X_y=True)
    ml_models["iris_predictor"] = LogisticRegression(random_state=0, max_iter=500).fit(X, y)
    yield
    # Clean up the ML models and release the resources
    ml_models.clear()
"""

md(f"```python\n{lifespan}\n```")

```python
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression

ml_models = {}

@asynccontextmanager
async def lifespan():
    # Load the ML model
    X, y = load_iris(return_X_y=True)
    ml_models["iris_predictor"] = LogisticRegression(random_state=0, max_iter=500).fit(X, y)
    yield
    # Clean up the ML models and release the resources
    ml_models.clear()

```

In [ ]:
app = """
from fastkafka import FastKafka

kafka_brokers = {
    "localhost": {
        "url": "localhost",
        "description": "local development kafka broker",
        "port": 9092,
    },
    "production": {
        "url": "kafka.airt.ai",
        "description": "production kafka broker",
        "port": 9092,
        "protocol": "kafka-secure",
        "security": {"type": "plain"},
    },
}

kafka_app = FastKafka(
    title="Iris predictions",
    kafka_brokers=kafka_brokers,
    lifespan=lifespan,
)

@kafka_app.consumes(topic="input_data", auto_offset_reset="latest")
async def on_input_data(msg: IrisInputData):
    global model
    species_class = ml_models["iris_predictor"].predict(
        [[msg.sepal_length, msg.sepal_width, msg.petal_length, msg.petal_width]]
    )[0]

    to_predictions(species_class)


@kafka_app.produces(topic="predictions")
def to_predictions(species_class: int) -> IrisPrediction:
    iris_species = ["setosa", "versicolor", "virginica"]

    prediction = IrisPrediction(species=iris_species[species_class])
    return prediction
"""

In [ ]:
complete_app = lifespan + app
md(f"```python\n{complete_app}\n```")

```python
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression

ml_models = {}

@asynccontextmanager
async def lifespan():
    # Load the ML model
    X, y = load_iris(return_X_y=True)
    ml_models["iris_predictor"] = LogisticRegression(random_state=0, max_iter=500).fit(X, y)
    yield
    # Clean up the ML models and release the resources
    ml_models.clear()

from fastkafka import FastKafka

kafka_brokers = {
    "localhost": {
        "url": "localhost",
        "description": "local development kafka broker",
        "port": 9092,
    },
    "production": {
        "url": "kafka.airt.ai",
        "description": "production kafka broker",
        "port": 9092,
        "protocol": "kafka-secure",
        "security": {"type": "plain"},
    },
}

kafka_app = FastKafka(
    title="Iris predictions",
    kafka_brokers=kafka_brokers,
    lifespan=lifespan,
)

@kafka_app.consumes(topic="input_data", auto_offset_reset="latest")
async def on_input_data(msg: IrisInputData):
    global model
    species_class = ml_models["iris_predictor"].predict(
        [[msg.sepal_length, msg.sepal_width, msg.petal_length, msg.petal_width]]
    )[0]

    to_predictions(species_class)


@kafka_app.produces(topic="predictions")
def to_predictions(species_class: int) -> IrisPrediction:
    iris_species = ["setosa", "versicolor", "virginica"]

    prediction = IrisPrediction(species=iris_species[species_class])
    return prediction

```